In [1]:
import pandas as pd
import numpy as np

import nltk
import string
import re
import json, requests

In [ ]:
df = pd.read_csv("../xtra/180725C_IR.csv")
df["year"] = df["year"].apply(int)
df["year"] = df["year"].apply(str)

In [ ]:
translator = str.maketrans('', '', string.punctuation)
def preprocess_new_line(text):
    text = "".join(text.splitlines())
    text =  text.translate(translator)
    return  " ".join(text.split())

In [ ]:
cols = ['lyricist', 'movie', 'song_name', 'year',
       'lyrics_tamil', 'music_director','lines', 'meanings', 'tags',
       'feels']
# for col in cols:
#     df[col] = df[col].apply(preprocess_new_line)
df["lyrics_tamil"] = df["lyrics_tamil"].apply(preprocess_new_line)

In [ ]:
json_data = {
    "version": "v3",
    "data" : []
}
for idx,row in df.iterrows():
    json_data["data"].append({
        "திரைப்படம்" : row["movie"],
        "பாடலாசிரியர்" : row["lyricist"],
        "இசையமைப்பாளர்" : row["music_director"],
        "பாடல்" : row["song_name"],
        "வருடம்" : row["year"],
        "பாடல்வரிகள்" : row["lyrics_tamil"],
        "உருவகம்" : row["lines"],
        "விளக்கம்": row["meanings"],
        "உணர்வு" : row["feels"],
        "குறிச்சொல்": row["tags"],
    })

In [ ]:
with open("../data/preprocessed_v1_725c.json", 'w', encoding='utf-8-sig') as fp:
    json.dump(json_data, fp, ensure_ascii=False)

In [ ]:
json_data

In [2]:
def agg_multi_match_and_sort_q(query, fields, operator="or", sort_num=10):
    q = {
        "size": sort_num,
        # "sort": [
        #     {"year": {"order": "desc"}},
        # ],
        "query": {
            "multi_match": {
                "query": query,
                "fields": fields,
                "operator": operator,
                "type": "best_fields",
            }
        },
        "aggs": {
            "Movie Filter": {"terms": {"field": "movie.keyword", "size": 10}},
            "Lyricist Filter": {"terms": {"field": "lyricist.keyword", "size": 10}},
            "Music_director Filter": {
                "terms": {"field": "music_director.keyword", "size": 10}
            },
            "Song Filter": {"terms": {"field": "song.keyword", "size": 10}},
        },
    }
    q = json.dumps(q)
    return q

In [5]:
q_code = agg_multi_match_and_sort_q(
    "அன்பே",
    ["movie", "song"]
)
response = requests.post(
    "http://localhost:9200/arrtamil2/_search",
    json=q_code,
)
json_data = response.json()
# hits = json_data["hits"]["hits"]

In [6]:
json_data

{'error': {'root_cause': [{'type': 'parsing_exception',
    'reason': 'Expected [START_OBJECT] but found [VALUE_STRING]',
    'line': 1,
    'col': 1}],
  'type': 'parsing_exception',
  'reason': 'Expected [START_OBJECT] but found [VALUE_STRING]',
  'line': 1,
  'col': 1},
 'status': 400}